In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier

In [4]:
#59714 is the output length of this full thing!
Ntrain,Ntest,Nsub =87141731,32225162,6245533

In [6]:
chunksize=50000 # Out of 87141731.
train = pd.read_csv('E:/kaggle_Outbrain_Click_Prediction/train_doc.csv.gz',iterator=True,chunksize=chunksize) #Load data

for chunk in train:
    predictors=[x for x in chunk.columns if x not in ['display_id','clicked']]
    chunk=chunk.fillna(0.0)
    alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
    alg.fit(chunk[predictors], chunk["clicked"])#Fit the Algorithm
 #   if testing:
 #       break

print('Done Training')

Done Training


In [7]:
del(train)

In [8]:
test_doc = pd.read_csv('E:/kaggle_Outbrain_Click_Prediction/test_doc.csv.gz',iterator=True,chunksize=chunksize) #Load data
predY=[]
for chunk in test_doc :
    init_chunk_size=len(chunk)
    chunk=chunk.fillna(0.0)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred
#    if testing:
#    break
    
print('Done Testing')

Done Testing


In [12]:
del(test_doc)

32225162

In [20]:
test = pd.read_csv('E:/kaggle_Outbrain_Click_Prediction/clicks_test.csv.zip')

In [21]:
results=pd.concat((test,pd.DataFrame(predY)) ,axis=1,ignore_index=True)#Combine the predicted values with the ids
print(results.head(10))

          0       1         2
0  16874594   66758  0.388889
1  16874594  150083  0.222222
2  16874594  162754  0.222222
3  16874594  170392  0.222222
4  16874594  172888  0.222222
5  16874594  180797  0.500000
6  16874595    8846  0.500000
7  16874595   30609  0.388889
8  16874595  143982  0.288889
9  16874596   11430  0.388889


In [22]:
results.columns = ['display_id','ad_id','clicked']#Rename the columns
#results=results[results['clicked'] > 0.0]
results = results.sort_values(by=['display_id','clicked'], ascending=[True, False])
results = results.reset_index(drop=True)
results=results[['display_id','ad_id']].groupby('display_id')['ad_id'].agg(lambda col: ' '.join(map(str,col)))
results.columns=[['display_id','ad_id']]

print(results.head())
print(results.shape)

display_id
16874594             180797 66758 150083 162754 170392 172888
16874595                                    8846 30609 143982
16874596    11430 57197 132820 288385 289122 289915 153260...
16874597    180965 182039 137858 143981 285834 305790 3088...
16874598                           67292 250082 145937 335632
Name: ad_id, dtype: object
(6245533,)


In [26]:
filename = 'E:/kaggle_Outbrain_Click_Prediction/TREE_n10_split4_leaf2_simpleRF'
results.to_csv(filename+'.csv', header = True)